In [1]:
!pip install yt-dlp


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
from yt_dlp import YoutubeDL
import cv2 
import matplotlib.pyplot as plt

In [7]:
url = "https://www.youtube.com/watch?v=SsZ-0AzryUg"
ydl_opts = {
    "format": "bestvideo",
    "outtmpl": "video_only2.%(ext)s"
}

In [8]:
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=SsZ-0AzryUg
[youtube] SsZ-0AzryUg: Downloading webpage
[youtube] SsZ-0AzryUg: Downloading android sdkless player API JSON
[youtube] SsZ-0AzryUg: Downloading tv client config
[youtube] SsZ-0AzryUg: Downloading tv player API JSON
[youtube] SsZ-0AzryUg: Downloading web safari player API JSON
[youtube] SsZ-0AzryUg: Downloading player e237d4c5-main


         player = https://www.youtube.com/s/player/e237d4c5/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] SsZ-0AzryUg: Downloading m3u8 information
[info] SsZ-0AzryUg: Downloading 1 format(s): 303
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: video_only2.webm
[download] 100.0% of   19.92MiB at    4.29MiB/s ETA 00:00  [download] Unable to rename file: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'video_only2.webm.part' -> 'video_only2.webm'. Retrying (1/3)...
[download] Unable to rename file: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'video_only2.webm.part' -> 'video_only2.webm'. Retrying (2/3)...
[download] 100% of   19.92MiB in 00:00:00 at 27.09MiB/s  


In [1]:
import subprocess

In [2]:
ffmpeg_path = r"D:\University\semestr_7\mmrokz\homework6\ffmpeg\ffmpeg-2025-11-02-git-f5eb11a71d-essentials_build\bin\ffmpeg.exe"

In [ ]:
input_file = "video_only.mp4"
output_file = "clip_fixed.mp4"
start_time = "00:03:00"
end_time   = "00:03:15"

command = [
    ffmpeg_path,
    "-ss", start_time,        
    "-to", end_time,          
    "-i", input_file,         
    "-c:v", "libx264",        
    "-movflags", "faststart",             
    "-y",                     
    output_file
]

subprocess.run(command)
print("✅ Відео збережено як", output_file)

✅ Відео збережено як clip_fixed.mp4


In [9]:
input_file = "video_only2.webm"
output_file = "clip_fixed2.mp4"
start_time = "00:00:00"
end_time   = "00:00:15"

command = [
    ffmpeg_path,
    "-ss", start_time,        
    "-to", end_time,          
    "-i", input_file,         
    "-c:v", "libx264",        
    "-c:a", "aac", 
    "-movflags", "faststart",             
    "-y",                     
    output_file
]

subprocess.run(command)
print("✅ Відео збережено як", output_file)

✅ Відео збережено як clip_fixed2.mp4
